In [14]:
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from datetime import datetime

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver', options=options)

search_query = input("Enter a search query: ")

driver.get(f"https://www.youtube.com/results?search_query={search_query}")

driver.implicitly_wait(3)
time.sleep(1.5)

driver.execute_script('window.scrollTo(0,800)')
time.sleep(3)

last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1.5)

    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break

    last_height = new_height
    time.sleep(1.5)

    try:
        driver.find_element_by_css_selector('#dismiss-button > a').click()
        time.sleep(1.5)
    except:
        pass

html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')

video_links = soup.find_all('a', {'class': 'yt-simple-endpoint style-scope ytd-video-renderer'})
video_data = []

for link in video_links:
    video_url = 'https://www.youtube.com' + link['href']
    
    # Get video publication date
    date_element = link.find_next('div', {'id': 'metadata-line'})
    if date_element:
        publication_date = date_element.get_text(strip=True).split('•')[-1].strip()
    else:
        publication_date = 'N/A'
    
    video_data.append({'Video URL': video_url, 'Publication Date': publication_date})

driver.quit()

# Create filename with keyword and current time
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"{search_query}_{current_time}.csv"

# Save data to the CSV file
fieldnames = ['Video URL', 'Publication Date']

with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(video_data)

print(f"Data saved to '{filename}' file.")


Enter a search query: 화이자
Data saved to '화이자_2023-07-12_16-31-50.csv' file.


In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pandas as pd

def scrape_youtube_comments(urls):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome('chromedriver', options=options)
    comments_data = []

    for url in urls:
        driver.get(url)
        driver.implicitly_wait(1.8)
        time.sleep(1)

        driver.execute_script('window.scrollTo(0,800)')
        time.sleep(1.8)

        last_height = driver.execute_script("return document.documentElement.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(1)
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            time.sleep(1)

            try:
                driver.find_element_by_css_selector('#dismiss-button > a').click()
                time.sleep(1)
            except:
                pass

        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')

        id_list = soup.select('div#header-author > h3 > #author-text > span')
        comment_list = soup.select('yt-formatted-string#content-text')
        date_list = soup.select('yt-formatted-string.published-time-text')

        for i in range(len(comment_list)):
            temp_id = id_list[i].text
            temp_id = temp_id.replace('\n','').replace('\t','').replace(' ','').strip()

            temp_comment = comment_list[i].text
            temp_comment = temp_comment.replace('\n','').replace('\t','').strip()

            temp_date = date_list[i].text.strip()

            comments_data.append({'URL': url, '아이디': temp_id, '댓글 내용': temp_comment, '게시날짜': temp_date})

    driver.quit()

    youtube_df = pd.DataFrame(comments_data)
    file_path = 'crawl_pfizer.csv'
    youtube_df.to_csv(file_path, encoding='UTF-8-sig', index=False)
    print("Comments scraped and saved to:", file_path)

urls = [   
    'https://www.youtube.com/watch?v=QRogcfRpeKQ&pp=ygUJ7ZmU7J207J6Q',
    'https://www.youtube.com/shorts/k_JiUh6o3No',
    'https://www.youtube.com/shorts/XMCtSZM0Tg8',
    'https://www.youtube.com/watch?v=kajrras76GE&pp=ygUJ7ZmU7J207J6Q',
    'https://www.youtube.com/watch?v=BJtqCNz9Y1A&pp=ygUJ7ZmU7J207J6Q',
    'https://www.youtube.com/watch?v=CXwBUT0HSyY&pp=ygUJ7ZmU7J207J6Q'
]

scrape_youtube_comments(urls)

Comments scraped and saved to: crawl_pfizer.csv


# 1~3년전 데이터만 추출

In [6]:
import pandas as pd

# 업로드된 파일명
uploaded_filename = 'new 모더나 댓글 크롤링.csv'

# 새로운 파일명 생성
new_filename = uploaded_filename.split('.')[0] + '_extract.csv'

# CSV 파일 읽기
df = pd.read_csv(uploaded_filename)

# '게시날짜' 열에서 '년 전'을 포함한 댓글만 선택
df_extract = df[df['게시날짜'].str.contains('년 전')]

# 새로운 파일로 저장 (인코딩: utf-8-sig)
df_extract.to_csv(new_filename, index=False, encoding='utf-8-sig')

print("추출된 데이터가 {} 파일로 저장되었습니다.".format(new_filename))


추출된 데이터가 new 모더나 댓글 크롤링_extract.csv 파일로 저장되었습니다.
